# Hospital Operations Efficiency & Resource Utilization — EDA

This notebook performs exploratory data analysis (EDA) on the hospital operations dataset. It connects to a local SQL Server, pulls the required tables, runs validation checks, and produces visualizations using **matplotlib**. The notebook is modular and intended to be run end-to-end in VS Code or Jupyter.

**Author:** Aman Kumar Singh

---

## 0. Requirements

Install required packages in your environment before running the notebook:

```bash
pip install pandas numpy matplotlib sqlalchemy pyodbc seaborn nbformat
```

If you plan to use SQLAlchemy + pyodbc on Windows with Windows Authentication, ensure the correct ODBC driver is installed (e.g., ODBC Driver 17 for SQL Server).

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text

sns.set(style='whitegrid')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

OUTPUT_DIR = 'outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)

pd.options.display.max_columns = 200
pd.options.display.float_format = '{:,.2f}'.format
print('Imports complete')

## 1. SQL Database Connection

Edit the connection string below to match your environment. Uses SQLAlchemy with pyodbc driver.

In [ ]:
# Edit these values for your environment
server = 'localhost'  # or 'localhost\\SQLEXPRESS'
database = 'HospitalOpsDB'  # change to your database name
driver = 'ODBC Driver 17 for SQL Server'

# Windows Trusted Connection example
connection_url = f"mssql+pyodbc://@{server}/{database}?driver={driver.replace(' ', '+')}&trusted_connection=yes"

print('Connection URL (sanitized):', connection_url)

try:
    engine = create_engine(connection_url)
    print('✅ SQLAlchemy engine created')
except Exception as e:
    print('❌ Could not create engine — check connection details')
    print(e)

## 2. Load core tables into pandas
We'll load the 8 core tables into DataFrames. If the table names differ, update the SQL accordingly.

In [ ]:
table_names = ['Hospitals','Departments','Staff','Patients','Admissions','Surgeries','ResourceUtilizationSnapshots','Financials']

df = {}
for t in table_names:
    try:
        df[t] = pd.read_sql_table(t, con=engine)
        print(f"Loaded {t}:", df[t].shape)
    except Exception as e:
        try:
            df[t] = pd.read_sql(f'SELECT * FROM {t}', con=engine)
            print(f"Loaded {t} via read_sql:", df[t].shape)
        except Exception as ex:
            print(f"Failed to load {t}:", ex)

for k in df:
    display(k, df[k].head())

## 3. Basic Validation & Integrity Checks
Run quick sanity checks similar to what we ran in SQL.

In [ ]:
# 3.1 Row counts
for t in table_names:
    if t in df:
        print(f"{t}: {len(df[t])}")

# 3.2 Null foreign keys and referential checks
print('\nReferential checks:')
if 'Departments' in df and 'Hospitals' in df:
    orphans = df['Departments'][~df['Departments']['HospitalID'].isin(df['Hospitals']['HospitalID'])]
    print('Orphan Departments:', len(orphans))

if 'Staff' in df and 'Departments' in df:
    orphans = df['Staff'][~df['Staff']['DeptID'].isin(df['Departments']['DeptID'])]
    print('Orphan Staff rows:', len(orphans))

# 3.3 Date logic
if 'Admissions' in df:
    bad_dates = df['Admissions'][pd.to_datetime(df['Admissions']['DischargeDate']) < pd.to_datetime(df['Admissions']['AdmissionDate'])]
    print('Admissions with discharge < admit:', len(bad_dates))

# 3.4 Range checks for utilization
if 'ResourceUtilizationSnapshots' in df:
    r = df['ResourceUtilizationSnapshots']
    print('BedOcc min/max:', r['BedOccupancyRate'].min(), r['BedOccupancyRate'].max())
    print('EquipUtil min/max:', r['EquipmentUtilizationRate'].min(), r['EquipmentUtilizationRate'].max())

## 4. Descriptive Statistics
Overview statistics for key tables.

In [ ]:
if 'Admissions' in df:
    print('Admissions summary:')
    display(df['Admissions'].describe(include='all'))

if 'Patients' in df:
    print('Patients age distribution:')
    display(df['Patients']['Age'].describe())

if 'Staff' in df:
    print('Staff by Role:')
    display(df['Staff'].groupby('Role').agg({'StaffID':'count','ExperienceYears':'mean','Salary':'mean'}).reset_index())

## 5. Operational Analysis
Visualizing key operational metrics: LOS distribution, Bed occupancy, Readmission rate, ER wait times.

In [ ]:
plt.figure(figsize=(8,5))
if 'Admissions' in df:
    los = df['Admissions']['LengthOfStay'].dropna()
    plt.hist(los, bins=range(0,31), alpha=0.8)
    plt.title('Length of Stay Distribution')
    plt.xlabel('Days')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'los_hist.png'))
    plt.show()

# Bed occupancy histogram
if 'ResourceUtilizationSnapshots' in df:
    plt.figure(figsize=(8,5))
    plt.hist(df['ResourceUtilizationSnapshots']['BedOccupancyRate'].dropna()*100, bins=30)
    plt.title('Bed Occupancy Rate (%) Distribution')
    plt.xlabel('Occupancy %')
    plt.ylabel('Snapshots')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'bed_occupancy_hist.png'))
    plt.show()

# Readmission rate
if 'Admissions' in df:
    readm_rate = df['Admissions']['ReadmissionFlag'].mean()*100
    print(f'Readmission rate: {readm_rate:.2f}%')

# ER wait time
if 'ResourceUtilizationSnapshots' in df:
    plt.figure(figsize=(8,5))
    plt.boxplot(df['ResourceUtilizationSnapshots']['ERWaitTimeMinutes'].dropna())
    plt.title('ER Wait Time (minutes) - Distribution')
    plt.ylabel('Minutes')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'er_wait_box.png'))
    plt.show()

## 6. Staff Efficiency
Analyze staff utilization and salary vs experience.

In [ ]:
if 'Staff' in df:
    staff_summary = df['Staff'].groupby('Role').agg(Count=('StaffID','count'), AvgExp=('ExperienceYears','mean'), AvgSalary=('Salary','mean')).reset_index()
    display(staff_summary)

    plt.figure(figsize=(8,5))
    sns.barplot(data=staff_summary, x='Role', y='AvgSalary')
    plt.title('Avg Salary by Role')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'avg_salary_role.png'))
    plt.show()

    plt.figure(figsize=(8,5))
    sns.boxplot(data=df['Staff'], x='Role', y='ExperienceYears')
    plt.title('Experience Years by Role')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'exp_box_role.png'))
    plt.show()

## 7. Patient & Admission Insights
Age, gender, admission types, and outcomes.

In [ ]:
if 'Patients' in df:
    plt.figure(figsize=(8,5))
    plt.hist(df['Patients']['Age'].dropna(), bins=30)
    plt.title('Patient Age Distribution')
    plt.xlabel('Age')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'age_dist.png'))
    plt.show()

if 'Admissions' in df:
    adm_by_type = df['Admissions']['AdmissionType'].value_counts(normalize=True)*100
    print('Admission types (%):')
    display(adm_by_type)

    outcome_los = df['Admissions'].groupby('Outcome')['LengthOfStay'].mean().reset_index()
    display(outcome_los)
    plt.figure(figsize=(8,5))
    sns.barplot(data=outcome_los, x='Outcome', y='LengthOfStay')
    plt.title('Avg LOS by Outcome')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'los_by_outcome.png'))
    plt.show()

## 8. Surgery Outcomes & Efficiency
Analyze surgery durations, complication rates, and surgeon workload.

In [ ]:
if 'Surgeries' in df:
    print('Surgeries summary:')
    display(df['Surgeries'].describe(include='all'))

    surg_by_type = df['Surgeries'].groupby('SurgeryType').agg(Count=('SurgeryID','count'), AvgDuration=('DurationMinutes','mean'), CompRate=('ComplicationFlag','mean')).reset_index()
    display(surg_by_type.sort_values('Count', ascending=False).head(10))

    plt.figure(figsize=(10,6))
    sns.barplot(data=surg_by_type.sort_values('AvgDuration', ascending=False).head(10), x='AvgDuration', y='SurgeryType')
    plt.title('Top 10 Surgery Types by Avg Duration')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'surg_top10_avgdur.png'))
    plt.show()

## 9. Resource Utilization Trends (time-series)
Aggregate snapshots by month and visualize trends for occupancy and ER wait time.

In [ ]:
if 'ResourceUtilizationSnapshots' in df:
    snaps = df['ResourceUtilizationSnapshots'].copy()
    snaps['SnapshotDate'] = pd.to_datetime(snaps['SnapshotDate'])
    snaps['Month'] = snaps['SnapshotDate'].dt.to_period('M').dt.to_timestamp()

    monthly = snaps.groupby('Month').agg(AvgBedOcc=('BedOccupancyRate','mean'), AvgERWait=('ERWaitTimeMinutes','mean')).reset_index()
    display(monthly.head())

    plt.figure(figsize=(10,5))
    plt.plot(monthly['Month'], monthly['AvgBedOcc']*100, marker='o')
    plt.title('Monthly Avg Bed Occupancy (%)')
    plt.xlabel('Month')
    plt.ylabel('Occupancy %')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'monthly_bedocc.png'))
    plt.show()

    plt.figure(figsize=(10,5))
    plt.plot(monthly['Month'], monthly['AvgERWait'], marker='o', color='orange')
    plt.title('Monthly Avg ER Wait (min)')
    plt.xlabel('Month')
    plt.ylabel('Minutes')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'monthly_erwait.png'))
    plt.show()

## 10. Financial Snapshot & Profitability
Analyze profit margins and insurance coverage.

In [ ]:
if 'Financials' in df:
    fin = df['Financials'].copy()
    fin['MonthStart'] = pd.to_datetime(fin['MonthStart'])
    profit_by_dept = fin.groupby('DeptID').agg(AvgProfitMargin=('ProfitMargin','mean'), AvgInsuranceCov=('InsuranceCoveragePct','mean')).reset_index()
    display(profit_by_dept.sort_values('AvgProfitMargin', ascending=False).head(10))

    plt.figure(figsize=(8,5))
    plt.hist(fin['ProfitMargin'].dropna(), bins=50)
    plt.title('Profit Margin Distribution')
    plt.xlabel('Profit Margin')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'profit_margin_hist.png'))
    plt.show()

## 11. Correlation & Feature Relationships
Compute correlations for numeric operational features and plot a heatmap.

In [ ]:
# Build a numeric features frame
num_frames = []
if 'Admissions' in df:
    num_frames.append(df['Admissions'][['LengthOfStay','ReadmissionFlag']])
if 'ResourceUtilizationSnapshots' in df:
    snaps = df['ResourceUtilizationSnapshots'][['BedOccupancyRate','EquipmentUtilizationRate','StaffUtilizationRate','ERWaitTimeMinutes']]
    num_frames.append(snaps)

if num_frames:
    num_df = pd.concat(num_frames, axis=1)
    num_df = num_df.select_dtypes(include=[np.number]).dropna()
    corr = num_df.corr()
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Correlation Heatmap (numeric features)')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,'corr_heatmap.png'))
    plt.show()
else:
    print('No numeric frames available for correlation')

## 12. Export aggregated CSVs for Power BI
Export light-weight aggregates that Power BI will use for dashboards (e.g., monthly occupancy, top high-LOS patients).

In [ ]:
# Example exports
if 'Admissions' in df:
    monthly_adm = df['Admissions'].copy()
    monthly_adm['AdmissionDate'] = pd.to_datetime(monthly_adm['AdmissionDate'])
    monthly_adm['Month'] = monthly_adm['AdmissionDate'].dt.to_period('M').dt.to_timestamp()
    agg_month = monthly_adm.groupby('Month').agg(TotalAdmissions=('AdmissionID','count'), AvgLOS=('LengthOfStay','mean')).reset_index()
    agg_month.to_csv(os.path.join(OUTPUT_DIR,'agg_monthly_admissions.csv'), index=False)
    print('Saved agg_monthly_admissions.csv')

if 'ResourceUtilizationSnapshots' in df:
    snaps = df['ResourceUtilizationSnapshots'].copy()
    snaps.to_csv(os.path.join(OUTPUT_DIR,'resource_snapshots_export.csv'), index=False)
    print('Saved resource_snapshots_export.csv')

if 'Admissions' in df:
    top_los = df['Admissions'].sort_values('LengthOfStay', ascending=False).head(100)
    top_los.to_csv(os.path.join(OUTPUT_DIR,'top100_los.csv'), index=False)
    print('Saved top100_los.csv')

## 13. Key Takeaways & Next Steps
- Summarize findings and propose next steps (predictive modeling for discharge delay, staff scheduling optimization, and Power BI dashboards).